<a href="https://colab.research.google.com/github/Lee-Sammy/Hyunseo_DTSC3020_Fall2025/blob/main/HyunseoLee_Assignment5_Ch_10%2611.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment5: CRM Cleanup @ **DalaShop**
*Files (Ch.10), Exceptions (Ch.10), Unit Tests (Ch.11), and Regular Expressions*  
.....

**Total: 3 points**  (Two questions, 1.5 pts each)  

> This assignment is scenario-based and aligned with Python Crash Course Ch.10 (files & exceptions), Ch.11 (unit testing with `unittest`), and Regular Expressions.

## Scenario
You are a data intern at an online retailer called **DalaShop**.  
Sales exported a **raw contacts** file from the CRM. It contains customer names, emails, and phone numbers, but the formatting is messy and some emails are invalid.  
Your tasks:

1. **Clean** the contacts (Files + Exceptions + Regex).  
2. **Write unit tests** to make sure your helper functions work correctly and keep working in the future.

## Data file (given by the company): `contacts_raw.txt`
Use this exact sample data (you may extend it for your own testing, but do **not** change it when submitting).  
Run the next cell once to create the file beside your notebook.

In [7]:
# Create the provided company dataset file
with open("contacts_raw.txt", "w", encoding="utf-8") as f:
    f.write('Alice Johnson <alice@example.com> , +1 (469) 555-1234\nBob Roberts <bob[at]example.com> , 972-555-777\nSara M. , sara@mail.co , 214 555 8888\n"Mehdi A." <mehdi.ay@example.org> , (469)555-9999\nDelaram <delaram@example.io>, +1-972-777-2121\nNima <NIMA@example.io> , 972.777.2121\nduplicate <Alice@Example.com> , 469 555 1234')
print("Wrote contacts_raw.txt with sample DalaShop data.")

Wrote contacts_raw.txt with sample DalaShop data.


## Q1 (1.5 pts) — CRM cleanup with Files, Exceptions, and Regex
Implement `q1_crm_cleanup.py` to:

1. **Read** `contacts_raw.txt` using `pathlib` and `with`. If the file is missing, **handle** it gracefully with `try/except FileNotFoundError` (print a friendly message; do not crash).
2. **Validate emails** with a simple regex (`r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}"`).  
   - Trim whitespace with `strip()` before checking.  
   - Use **full** matching (not partial).
3. **Normalize phone numbers:** remove all non-digits (e.g., with `re.sub(r"\D", "", raw)`).  
   - If the result has **≥ 10 digits**, keep the **last 10 digits**.  
   - Otherwise, return an **empty string** (`""`).
4. **Filter rows:** keep **only** rows with a valid email.
5. **Deduplicate:** remove duplicates by **email** using **case-insensitive** comparison (e.g., `email.casefold()`). **Keep the first occurrence** and drop later duplicates.
6. **Output CSV:** write to `contacts_clean.csv` with **columns exactly** `name,email,phone` (UTF-8).  
7. **Preserve input order:** the order of rows in `contacts_clean.csv` must match the **first appearance** order from the input file. **Do not sort** the rows.

**Grading rubric (1.5 pts):**
- (0.4) File read/write via `pathlib` + graceful `FileNotFoundError` handling  
- (0.5) Correct email regex validation + filtering  
- (0.4) Phone normalization + case-insensitive de-dup (keep first)  
- (0.2) Clean code, clear names, minimal docstrings/comments

In [12]:
code = """from pathlib import Path
import re
import csv

EMAIL_RE = re.compile(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\\.[A-Za-z]{2,}")

def is_valid_email(raw_email):
    if not raw_email:
        return ""
    email = raw_email.strip()
    if EMAIL_RE.fullmatch(email):
        return email
    return ""

def normalize_phone(raw_phone):
    if not raw_phone:
        return ""
    digits = re.sub(r"\\D", "", raw_phone)
    if len(digits) >= 10:
        return digits[-10:]
    return ""

def parse_contact_line(line):
    parts = line.strip().split(",")
    name = ""
    email_raw = ""
    phone_raw = ""
    if len(parts) == 1:
        left = parts[0].strip()
        phone_raw = ""
    elif len(parts) == 2:
        left = parts[0].strip()
        phone_raw = parts[1].strip()
    else:
        left = parts[0].strip()
        email_raw = parts[1].strip()
        phone_raw = parts[2].strip()
    if "<" in left and ">" in left:
        before, after = left.split("<", 1)
        name = before.strip().strip('"')
        email_raw = after.split(">", 1)[0].strip()
    else:
        name = left.strip().strip('"')
    email = is_valid_email(email_raw)
    phone = normalize_phone(phone_raw)
    return name, email, phone

def load_contacts(path: Path):
    rows = []
    try:
        text = path.read_text(encoding="utf-8")
    except FileNotFoundError:
        print("contacts_raw.txt not found. Please make sure the file exists.")
        return rows
    for line in text.splitlines():
        if not line.strip():
            continue
        name, email, phone = parse_contact_line(line)
        rows.append((name, email, phone))
    return rows

def dedupe_and_filter(rows):
    cleaned = []
    seen = set()
    for name, email, phone in rows:
        if not email:
            continue
        key = email.casefold()
        if key in seen:
            continue
        seen.add(key)
        cleaned.append((name, email, phone))
    return cleaned

def write_contacts_csv(path: Path, rows):
    with path.open("w", encoding="utf-8", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["name", "email", "phone"])
        for name, email, phone in rows:
            writer.writerow([name, email, phone])

def main():
    in_path = Path("contacts_raw.txt")
    out_path = Path("contacts_clean.csv")
    rows = load_contacts(in_path)
    rows = dedupe_and_filter(rows)
    write_contacts_csv(out_path, rows)
    print("Wrote contacts_clean.csv")

if __name__ == "__main__":
    main()
"""

with open("q1_crm_cleanup.py", "w", encoding="utf-8") as f:
    f.write(code)

print("saved q1_crm_cleanup.py")


saved q1_crm_cleanup.py


In [14]:
!python3 q1_crm_cleanup.py
!cat contacts_clean.csv

Wrote contacts_clean.csv
name,email,phone
Alice Johnson,alice@example.com,4695551234
Sara M.,sara@mail.co,2145558888
Mehdi A.,mehdi.ay@example.org,4695559999
Delaram,delaram@example.io,9727772121
Nima,NIMA@example.io,9727772121


## Q2 (1.5 pts) — Unit testing with `unittest`
Create tests in `test_crm_cleanup.py` that cover at least:

1. **Email validation**: valid/invalid variations.  
2. **Phone normalization**: parentheses, dashes, spaces, country code; too-short cases.  
3. **Parsing**: from a small multi-line string (not from a file), assert the exact structured rows (name/email/phone).  
4. **De-duplication**: demonstrate that a case-variant duplicate email is dropped (first occurrence kept).




In [17]:
test_code = """import unittest
from q1_crm_cleanup import (
    is_valid_email,
    normalize_phone,
    parse_contact_line,
    dedupe_and_filter,
)

class TestCRMCleanup(unittest.TestCase):
    def test_valid_email(self):
        self.assertEqual(is_valid_email("alice@example.com"), "alice@example.com")
        self.assertEqual(is_valid_email("USER+1@mail.co"), "USER+1@mail.co")

    def test_invalid_email(self):
        self.assertEqual(is_valid_email("bob[at]example.com"), "")
        self.assertEqual(is_valid_email("no-at-sign.com"), "")
        self.assertEqual(is_valid_email("   bad@@mail.com   "), "")

    def test_phone_normalization(self):
        self.assertEqual(normalize_phone("(469) 555-1234"), "4695551234")
        self.assertEqual(normalize_phone("+1 (469) 555-1234"), "4695551234")
        self.assertEqual(normalize_phone("972.777.2121"), "9727772121")
        self.assertEqual(normalize_phone("214 555 8888"), "2145558888")

    def test_phone_too_short(self):
        self.assertEqual(normalize_phone("972-555"), "")

    def test_parse_two_column_style(self):
        line = 'Alice Johnson <alice@example.com> , +1 (469) 555-1234'
        name, email, phone = parse_contact_line(line)
        self.assertEqual(name, "Alice Johnson")
        self.assertEqual(email, "alice@example.com")
        self.assertEqual(phone, "4695551234")

    def test_parse_three_column_style(self):
        line = "Sara M. , sara@mail.co , 214 555 8888"
        name, email, phone = parse_contact_line(line)
        self.assertEqual(name, "Sara M.")
        self.assertEqual(email, "sara@mail.co")
        self.assertEqual(phone, "2145558888")

    def test_parse_quoted_name(self):
        line = '"Mehdi A." <mehdi.ay@example.org> , (469)555-9999'
        name, email, phone = parse_contact_line(line)
        self.assertEqual(name, "Mehdi A.")
        self.assertEqual(email, "mehdi.ay@example.org")
        self.assertEqual(phone, "4695559999")

    def test_dedupe_case_insensitive(self):
        rows = [
            ("Alice Johnson", "alice@example.com", "4695551234"),
            ("duplicate", "Alice@Example.com", "4695551234"),
            ("Bob", "bob@example.com", "9725557777"),
        ]
        cleaned = dedupe_and_filter(rows)
        self.assertEqual(len(cleaned), 2)
        self.assertEqual(cleaned[0][0], "Alice Johnson")
        self.assertEqual(cleaned[1][0], "Bob")

    def test_dedupe_skips_empty_email(self):
        rows = [
            ("No Email", "", "1111111111"),
            ("Has Email", "ok@example.com", "2222222222"),
        ]
        cleaned = dedupe_and_filter(rows)
        self.assertEqual(len(cleaned), 1)
        self.assertEqual(cleaned[0][0], "Has Email")

if __name__ == "__main__":
    unittest.main()
"""

with open("test_crm_cleanup.py", "w") as f:
    f.write(test_code)

print("test_crm_cleanup.py saved")


test_crm_cleanup.py saved


In [18]:
!python -m unittest test_crm_cleanup.py


.........
----------------------------------------------------------------------
Ran 9 tests in 0.001s

OK


## Grading rubric (total 3 pts)
- **Q1 (1.5 pts)**  
  - (0.4) File I/O with `pathlib` + graceful `FileNotFoundError` handling  
  - (0.5) Email validation (regex + strip + full match) and filtering  
  - (0.4) Phone normalization and **case-insensitive** de-duplication (keep first)  
  - (0.2) Code clarity (names, minimal docstrings/comments)
- **Q2 (1.5 pts)**  
  - (0.6) Meaningful coverage for email/phone functions (valid & invalid)  
  - (0.6) Parsing & de-dup tests that assert exact expected rows  
  - (0.3) Standard `unittest` structure and readable test names
